<a href="https://colab.research.google.com/github/2403a52233-sudo/NLP_/blob/main/2403A52233_Assignment6_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1:LDA with Sample Data and BoW

Load text data

In [68]:
import pandas as pd

data=pd.read_excel("/content/LDA-Data.xlsx")

In [69]:
display(data)

,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [70]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Text Preprocesing: Clean Text, Word Tokenization, stopword removal, lemma,Rejoin

In [71]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import re

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

print("Preprocessing function `preprocess_text` defined.")

Preprocessing function `preprocess_text` defined.


In [72]:
data['preprocessed_news'] = data['News'].apply(preprocess_text)
display(data[['News', 'preprocessed_news']])

,News,preprocessed_news
0,Virat scored century in match,virat scored century match
1,BJP won in elections,bjp election
2,Bumra took 5 wicket in a match,bumra took wicket match
3,Congress form state government,congress form state government


## Create Bag-of-Words (BoW)




In [73]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    stop_words='english',
    max_df=0.95,
    min_df=1
)

doc_term_matrix = vectorizer.fit_transform(data['preprocessed_news'])
print(pd.DataFrame(doc_term_matrix.toarray(),
                   columns=vectorizer.get_feature_names_out()).head())

   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


## Convert BoW to DataFrame



In [74]:
feature_names = vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(), columns=feature_names)
display(bow_df.head())

,bjp,bumra,century,congress,election,form,government,match,scored,state,took,virat,wicket
0,0,0,1,0,0,0,0,1,1,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,1,0,1
3,0,0,0,1,0,1,1,0,0,1,0,0,0


## Apply LDA




In [75]:
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate LDA model
lda_model = LatentDirichletAllocation(
    n_components=2,  # Example: 2 topics
    random_state=42, # for reproducibility
    learning_method='batch' # suitable for smaller datasets
)

# Fit LDA model to the Bag-of-Words matrix and transform it
lda_output = lda_model.fit_transform(doc_term_matrix)

print(f"Shape of LDA output (document-topic distribution): {lda_output.shape}")

Shape of LDA output (document-topic distribution): (4, 2)


## Identify Words for Each Topic




In [76]:
n_top_words = 10# Define the number of top words to display for each topic

feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx + 1}:")
    # Sort the words by their probability in this topic and get the indices of the top words
    top_word_indices = topic.argsort()[:-n_top_words - 1:-1]
    top_words = [feature_names[i] for i in top_word_indices]
    print(f"{', '.join(top_words)}")

Topic 1:
form, government, congress, state, election, bjp, match, wicket, bumra, took
Topic 2:
match, virat, century, scored, took, bumra, wicket, bjp, election, state


In [77]:
# Assign topics to each document
document_topics = lda_model.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['preprocessed_news', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                preprocessed_news  topic
0      virat scored century match      1
1                    bjp election      0
2         bumra took wicket match      1
3  congress form state government      0


Task 2:LDA with Kaggle Data and BoW



Load text data

In [78]:
import pandas as pd

data=pd.read_csv("/content/arxiv_data.csv")

In [79]:
display(data)

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"
...,...,...,...
51769,Hierarchically-coupled hidden Markov models fo...,We address the problem of analyzing sets of no...,"['stat.ML', 'physics.bio-ph', 'q-bio.QM']"
51770,Blinking Molecule Tracking,We discuss a method for tracking individual mo...,"['cs.CV', 'cs.DM']"
51771,Towards a Mathematical Foundation of Immunolog...,We attempt to set a mathematical foundation of...,"['stat.ML', 'cs.LG', 'q-bio.GN']"
51772,A Semi-Automatic Graph-Based Approach for Dete...,Diffusion Tensor Imaging (DTI) allows estimati...,['cs.CV']


In [80]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Text Preprocesing: Clean Text, Word Tokenization, stopword removal, lemma,Rejoin

In [81]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import re

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

print("Preprocessing function `preprocess_text` defined.")

Preprocessing function `preprocess_text` defined.


In [82]:
data['preprocessed_news'] = data['titles'].apply(preprocess_text)
display(data[['titles', 'preprocessed_news']])

,titles,preprocessed_news
0,Survey on Semantic Stereo Matching / Semantic ...,survey semantic stereo matching semantic depth...
1,FUTURE-AI: Guiding Principles and Consensus Re...,futureai guiding principle consensus recommend...
2,Enforcing Mutual Consistency of Hard Regions f...,enforcing mutual consistency hard region semis...
3,Parameter Decoupling Strategy for Semi-supervi...,parameter decoupling strategy semisupervised l...
4,Background-Foreground Segmentation for Interio...,backgroundforeground segmentation interior sen...
...,...,...
51769,Hierarchically-coupled hidden Markov models fo...,hierarchicallycoupled hidden markov model lear...
51770,Blinking Molecule Tracking,blinking molecule tracking
51771,Towards a Mathematical Foundation of Immunolog...,towards mathematical foundation immunology ami...
51772,A Semi-Automatic Graph-Based Approach for Dete...,semiautomatic graphbased approach determining ...


## Create Bag-of-Words (BoW)




In [83]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    stop_words='english',
    max_df=0.95,
    min_df=1
)

doc_term_matrix = vectorizer.fit_transform(data['preprocessed_news'])
print(pd.DataFrame(doc_term_matrix.toarray(),
                   columns=vectorizer.get_feature_names_out()).head())

   aa  aaa  aabo  aacp  aad  aadnet  aads  aaformer  aag  aam  ...  \
0   0    0     0     0    0       0     0         0    0    0  ...   
1   0    0     0     0    0       0     0         0    0    0  ...   
2   0    0     0     0    0       0     0         0    0    0  ...   
3   0    0     0     0    0       0     0         0    0    0  ...   
4   0    0     0     0    0       0     0         0    0    0  ...   

   zoomtoinpaint  zoonosis  zooplankton  zootuning  zope  zorro  zpd  zscores  \
0              0         0            0          0     0      0    0        0   
1              0         0            0          0     0      0    0        0   
2              0         0            0          0     0      0    0        0   
3              0         0            0          0     0      0    0        0   
4              0         0            0          0     0      0    0        0   

   zsslr  zstgan  
0      0       0  
1      0       0  
2      0       0  
3      0       0

## Convert BoW to DataFrame





In [84]:
feature_names = vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(doc_term_matrix.toarray(), columns=feature_names)
display(bow_df.head())

,aa,aaa,aabo,aacp,aad,aadnet,aads,aaformer,aag,aam,...,zoomtoinpaint,zoonosis,zooplankton,zootuning,zope,zorro,zpd,zscores,zsslr,zstgan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Apply LDA




In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate LDA model
lda_model = LatentDirichletAllocation(
    n_components=2,  # Example: 2 topics
    random_state=42, # for reproducibility
    learning_method='batch' # suitable for smaller datasets
)

# Fit LDA model to the Bag-of-Words matrix and transform it
lda_output = lda_model.fit_transform(doc_term_matrix)

print(f"Shape of LDA output (document-topic distribution): {lda_output.shape}")

## Identify Words for Each Topic




In [ ]:
n_top_words = 10# Define the number of top words to display for each topic

feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx + 1}:")
    # Sort the words by their probability in this topic and get the indices of the top words
    top_word_indices = topic.argsort()[:-n_top_words - 1:-1]
    top_words = [feature_names[i] for i in top_word_indices]
    print(f"{', '.join(top_words)}")

Topic 1:
learning, graph, reinforcement, representation, model, image, network, detection, deep, feature
Topic 2:
network, image, deep, neural, using, learning, detection, adversarial, object, segmentation


In [33]:
# Assign topics to each document
# Re-initialize and fit LDA model to ensure it's configured for the current doc_term_matrix
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate LDA model (using the same parameters as in mWN-_rH_L6o7)
lda_model = LatentDirichletAllocation(
    n_components=2,  # Example: 2 topics
    random_state=42, # for reproducibility
    learning_method='batch' # suitable for smaller datasets
)

# Fit LDA model to the Bag-of-Words matrix
# This will update lda_model to expect 22945 features, matching doc_term_matrix
lda_model.fit(doc_term_matrix)

document_topics = lda_model.transform(doc_term_matrix)
data['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(data[['preprocessed_news', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                                   preprocessed_news  topic
0  survey semantic stereo matching semantic depth...      1
1  futureai guiding principle consensus recommend...      1
2  enforcing mutual consistency hard region semis...      0
3  parameter decoupling strategy semisupervised l...      0
4  backgroundforeground segmentation interior sen...      1


NMF with Sample Data and BoW

Load text data

In [34]:
import pandas as pd
df = pd.read_excel('/content/LDA-Data.xlsx')


In [35]:
display(df.head())


,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [36]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    # Initialize NLTK tools
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # 1. Preprocess text (from previous steps)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    text = text.lower()  # Convert to lowercase

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces

    # 2. Word Tokenization
    tokenized_words = word_tokenize(text)

    # 3. Stopword Removal
    filtered_words = [word for word in tokenized_words if word not in stop_words]

    # 4. Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # 5. Rejoin words
    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
df['clean_summaries_pipeline'] = df['News'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(df[['clean_summaries_pipeline']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
         clean_summaries_pipeline
0      virat scored century match
1                    bjp election
2       bumra took 5 wicket match
3  congress form state government


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
# Vectorize the cleaned summaries
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(df['clean_summaries_pipeline'])

In [39]:
import pandas as pd
# Get feature (word) names
feature_names = count_vectorizer.get_feature_names_out()
# Convert sparse matrix to DataFrame
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
# Display BoW matrix for top 10 documents
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   match
0      1
1      0
2      1
3      0


In [40]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize and fit LDA model
num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)


LatentDirichletAllocation(n_components=2, random_state=42)

In [41]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")

        # Get word weights for this topic
        topic_weights = model.components_[topic_idx]

        # Get indices of words sorted by weight (descending)
        sorted_indices = topic_weights.argsort()[::-1]

        # Take top N words
        top_indices = sorted_indices[:num_top_words]

        # Print top words
        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [42]:
# Display top words for each topic
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
match 

Topic 1:
match 


In [43]:
# Assign topics to each document
document_topics = LDA.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
         clean_summaries_pipeline  topic
0      virat scored century match      0
1                    bjp election      0
2       bumra took 5 wicket match      0
3  congress form state government      0


In [44]:
from sklearn.decomposition import NMF

# Initialize NMF model
num_topics = 2
nmf_model = NMF(n_components=num_topics, random_state=42)

# Fit the NMF model to the document-term matrix
nmf_model.fit(doc_term_matrix)
print("NMF model initialized and fitted successfully.")

NMF model initialized and fitted successfully.


In [45]:
num_top_words = 10
print(f"\nTop {num_top_words} words per topic (NMF):")
display_topics(nmf_model, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic (NMF):

Topic 0:
match 

Topic 1:
match 


In [46]:
# Assign topics to each document
document_topics = nmf_model.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
         clean_summaries_pipeline  topic
0      virat scored century match      0
1                    bjp election      0
2       bumra took 5 wicket match      0
3  congress form state government      0


 NMF with Kaggle Data and BoW



Load text data

In [47]:
import pandas as pd
df = pd.read_csv('/content/arxiv_data.csv')


In [48]:
display(df)

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"
...,...,...,...
51769,Hierarchically-coupled hidden Markov models fo...,We address the problem of analyzing sets of no...,"['stat.ML', 'physics.bio-ph', 'q-bio.QM']"
51770,Blinking Molecule Tracking,We discuss a method for tracking individual mo...,"['cs.CV', 'cs.DM']"
51771,Towards a Mathematical Foundation of Immunolog...,We attempt to set a mathematical foundation of...,"['stat.ML', 'cs.LG', 'q-bio.GN']"
51772,A Semi-Automatic Graph-Based Approach for Dete...,Diffusion Tensor Imaging (DTI) allows estimati...,['cs.CV']


In [49]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def nltk_preprocessing_pipeline(text):
    # Initialize NLTK tools
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # 1. Preprocess text (from previous steps)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    text = text.lower()  # Convert to lowercase

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces

    # 2. Word Tokenization
    tokenized_words = word_tokenize(text)

    # 3. Stopword Removal
    filtered_words = [word for word in tokenized_words if word not in stop_words]

    # 4. Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # 5. Rejoin words
    clean_summary = ' '.join(lemmatized_words)

    return clean_summary

print("NLTK preprocessing pipeline function created successfully!")

NLTK preprocessing pipeline function created successfully!


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [50]:
df['clean_summaries_pipeline'] = df['titles'].apply(nltk_preprocessing_pipeline)
print("\nComparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):")
print(df[['clean_summaries_pipeline']].head())


Comparison of previous clean_summaries and new clean_summaries_pipeline (first 5 rows):
                            clean_summaries_pipeline
0  survey semantic stereo matching semantic depth...
1  futureai guiding principle consensus recommend...
2  enforcing mutual consistency hard region semis...
3  parameter decoupling strategy semisupervised 3...
4  backgroundforeground segmentation interior sen...


In [51]:
from sklearn.feature_extraction.text import CountVectorizer
# Vectorize the cleaned summaries
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(df['clean_summaries_pipeline'])

In [52]:
import pandas as pd
# Get feature (word) names
feature_names = count_vectorizer.get_feature_names_out()
# Convert sparse matrix to DataFrame
bow_df = pd.DataFrame(doc_term_matrix.toarray(),columns=feature_names)
# Display BoW matrix for top 10 documents
bow_top_10 = bow_df.head(10)
print(bow_top_10)

   01  10  100  10x  11k  12lead  15  16x16  1d  1diabetes  ...  zhu  zigzag  \
0   0   0    0    0    0       0   0      0   0          0  ...    0       0   
1   0   0    0    0    0       0   0      0   0          0  ...    0       0   
2   0   0    0    0    0       0   0      0   0          0  ...    0       0   
3   0   0    0    0    0       0   0      0   0          0  ...    0       0   
4   0   0    0    0    0       0   0      0   0          0  ...    0       0   
5   0   0    0    0    0       0   0      0   0          0  ...    0       0   
6   0   0    0    0    0       0   0      0   0          0  ...    0       0   
7   0   0    0    0    0       0   0      0   0          0  ...    0       0   
8   0   0    0    0    0       0   0      0   0          0  ...    0       0   
9   0   0    0    0    0       0   0      0   0          0  ...    0       0   

   zone  zoo  zoom  zoomin  zooming  zoomintocheck  zoomnet  zootuning  
0     0    0     0       0        0           

In [53]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize and fit LDA model
num_topics = 2
LDA = LatentDirichletAllocation(n_components=num_topics, random_state=42)
LDA.fit(doc_term_matrix)


LatentDirichletAllocation(n_components=2, random_state=42)

In [54]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx in range(len(model.components_)):
        print(f"\nTopic {topic_idx}:")

        # Get word weights for this topic
        topic_weights = model.components_[topic_idx]

        # Get indices of words sorted by weight (descending)
        sorted_indices = topic_weights.argsort()[::-1]

        # Take top N words
        top_indices = sorted_indices[:num_top_words]

        # Print top words
        for idx in top_indices:
            print(feature_names[idx], end=" ")
        print()

In [55]:
# Display top words for each topic
num_top_words = 10
print(f"\nTop {num_top_words} words per topic:")
display_topics(LDA, count_vectorizer.get_feature_names_out(), num_top_words)


Top 10 words per topic:

Topic 0:
learning deep reinforcement representation model using recognition graph neural visual 

Topic 1:
network image detection segmentation object neural graph using adversarial 3d 


In [56]:
# Assign topics to each document
document_topics = LDA.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                            clean_summaries_pipeline  topic
0  survey semantic stereo matching semantic depth...      1
1  futureai guiding principle consensus recommend...      0
2  enforcing mutual consistency hard region semis...      1
3  parameter decoupling strategy semisupervised 3...      1
4  backgroundforeground segmentation interior sen...      1


In [57]:
# Assign topics to each document
from sklearn.decomposition import NMF

# Re-initialize and fit NMF model with the correct doc_term_matrix
num_topics = 2 # Assuming 2 topics, consistent with previous LDA example
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(doc_term_matrix)

document_topics = nmf_model.transform(doc_term_matrix)
df['topic'] = document_topics.argmax(axis=1)

print("\nDataFrame with assigned topics (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic']].head())


DataFrame with assigned topics (first 5 rows):
                            clean_summaries_pipeline  topic
0  survey semantic stereo matching semantic depth...      1
1  futureai guiding principle consensus recommend...      1
2  enforcing mutual consistency hard region semis...      1
3  parameter decoupling strategy semisupervised 3...      1
4  backgroundforeground segmentation interior sen...      1


Task 5: NMF with Sample Data and TFIDF



In [58]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd # Ensure pandas is imported for DataFrame display

# Create a CountVectorizer specifically for the Kaggle dataset
kaggle_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2,
    stop_words='english'
)

# Fit and transform the clean_summaries_pipeline column from the Kaggle DataFrame
kaggle_doc_term_matrix = kaggle_vectorizer.fit_transform(df['clean_summaries_pipeline'])

print("sample doc-term matrix created successfully.")
print(f"Shape of sample doc-term matrix: {kaggle_doc_term_matrix.shape}")

# Display the first few rows of the doc-term matrix
print("\nFirst 5 rows of the Kaggle doc-term matrix (dense representation):")
print(pd.DataFrame(kaggle_doc_term_matrix.toarray(), columns=kaggle_vectorizer.get_feature_names_out()).head())

sample doc-term matrix created successfully.
Shape of sample doc-term matrix: (51774, 12202)

First 5 rows of the Kaggle doc-term matrix (dense representation):
   01  10  100  10x  11k  12lead  15  16x16  1d  1diabetes  ...  zhu  zigzag  \
0   0   0    0    0    0       0   0      0   0          0  ...    0       0   
1   0   0    0    0    0       0   0      0   0          0  ...    0       0   
2   0   0    0    0    0       0   0      0   0          0  ...    0       0   
3   0   0    0    0    0       0   0      0   0          0  ...    0       0   
4   0   0    0    0    0       0   0      0   0          0  ...    0       0   

   zone  zoo  zoom  zoomin  zooming  zoomintocheck  zoomnet  zootuning  
0     0    0     0       0        0              0        0          0  
1     0    0     0       0        0              0        0          0  
2     0    0     0       0        0              0        0          0  
3     0    0     0       0        0              0        0       

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95,
    min_df=1,
    stop_words='english'
)

# Fit and transform the preprocessed news data to create the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(df['clean_summaries_pipeline'])

print("TF-IDF matrix created successfully.")
print(f"Shape of TF-IDF matrix: {tfidf_matrix.shape}")

TF-IDF matrix created successfully.
Shape of TF-IDF matrix: (51774, 23208)


In [60]:
from sklearn.decomposition import NMF

# Initialize NMF model
num_topics_tfidf = 2 # Keeping the number of topics consistent for now
nmf_model_tfidf = NMF(n_components=num_topics_tfidf, random_state=42)

# Fit the NMF model to the TF-IDF matrix
nmf_model_tfidf.fit(tfidf_matrix)

print("NMF model with TF-IDF initialized and fitted successfully.")

NMF model with TF-IDF initialized and fitted successfully.


In [61]:
n_top_words_tfidf = 10

print(f"\nTop {n_top_words_tfidf} words per topic (NMF with TF-IDF):")
display_topics(nmf_model_tfidf, tfidf_vectorizer.get_feature_names_out(), n_top_words_tfidf)


Top 10 words per topic (NMF with TF-IDF):

Topic 0:
network neural graph convolutional adversarial image generative detection using object 

Topic 1:
learning reinforcement deep representation unsupervised transfer model image using data 


In [62]:
# Assign topics to each document based on NMF with TF-IDF
document_topics_tfidf = nmf_model_tfidf.transform(tfidf_matrix)
df['topic_nmf_tfidf'] = document_topics_tfidf.argmax(axis=1)

print("\nDataFrame with assigned topics from NMF with TF-IDF (first 5 rows):")
print(df[['clean_summaries_pipeline', 'topic_nmf_tfidf']].head())


DataFrame with assigned topics from NMF with TF-IDF (first 5 rows):
                            clean_summaries_pipeline  topic_nmf_tfidf
0  survey semantic stereo matching semantic depth...                0
1  futureai guiding principle consensus recommend...                1
2  enforcing mutual consistency hard region semis...                0
3  parameter decoupling strategy semisupervised 3...                0
4  backgroundforeground segmentation interior sen...                0


Task 6:NMF with Kaggle Data and TFIDF



In [63]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a CountVectorizer specifically for the Kaggle dataset
kaggle_vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2,
    stop_words='english'
)

# Fit and transform the clean_summaries_pipeline column from the Kaggle DataFrame
kaggle_doc_term_matrix = kaggle_vectorizer.fit_transform(data['preprocessed_news'])

print("sample doc-term matrix created successfully.")
print(f"Shape of sample doc-term matrix: {kaggle_doc_term_matrix.shape}")

sample doc-term matrix created successfully.
Shape of sample doc-term matrix: (51774, 12125)


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95,
    min_df=1,
    stop_words='english'
)

# Fit and transform the preprocessed news data to create the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(data['preprocessed_news'])

print("TF-IDF matrix created successfully.")
print(f"Shape of TF-IDF matrix: {tfidf_matrix.shape}")

TF-IDF matrix created successfully.
Shape of TF-IDF matrix: (51774, 22945)


In [65]:
from sklearn.decomposition import NMF

# Initialize NMF model
num_topics_tfidf = 2 # Keeping the number of topics consistent for now
nmf_model_tfidf = NMF(n_components=num_topics_tfidf, random_state=42)

# Fit the NMF model to the TF-IDF matrix
nmf_model_tfidf.fit(tfidf_matrix)

print("NMF model with TF-IDF initialized and fitted successfully.")

NMF model with TF-IDF initialized and fitted successfully.


In [66]:
n_top_words_tfidf = 10

print(f"\nTop {n_top_words_tfidf} words per topic (NMF with TF-IDF):")
display_topics(nmf_model_tfidf, tfidf_vectorizer.get_feature_names_out(), n_top_words_tfidf)


Top 10 words per topic (NMF with TF-IDF):

Topic 0:
network neural graph convolutional adversarial generative image detection using object 

Topic 1:
learning reinforcement deep representation unsupervised transfer image using model data 


In [67]:
# Assign topics to each document based on NMF with TF-IDF
document_topics_tfidf = nmf_model_tfidf.transform(tfidf_matrix)
data['topic_nmf_tfidf'] = document_topics_tfidf.argmax(axis=1)

print("\nDataFrame with assigned topics from NMF with TF-IDF (first 5 rows):")
print(data[['preprocessed_news', 'topic_nmf_tfidf']].head())


DataFrame with assigned topics from NMF with TF-IDF (first 5 rows):
                                   preprocessed_news  topic_nmf_tfidf
0  survey semantic stereo matching semantic depth...                1
1  futureai guiding principle consensus recommend...                1
2  enforcing mutual consistency hard region semis...                0
3  parameter decoupling strategy semisupervised l...                1
4  backgroundforeground segmentation interior sen...                0
